<img src="https://github.com/Unidata/MetPy/raw/main/docs/_static/metpy_150x150.png" width=150 alt="MetPy logo"></img>
<img src="https://github.com/Unidata/MetPy/raw/main/docs/_static/unidata_150x150.png" width=150 alt="Unidata logo"></img>

# Units and xarray

---

## Overview
The core of this presentation is to introduce to you the required usage of the [Pint](https://pint.readthedocs.io/en/stable/)-based `metpy.units` for making MetPy's calculations unit-aware and correct! Understanding Pint, while required, can also enable for you a lot of convenience and sparing of headaches moving forward.

Similarly, MetPy now heavily relies on [xarray](https://docs.xarray.dev/en/stable/). For some functionality, xarray is required. However, for most of MetPy, xarray instead can be used for convenience and speed in your day-to-day science.

---

## Imports

In [1]:
import metpy.calc as mpcalc
import numpy as np
import xarray as xr

# get_test_data is primarily for internal testing and documentation
from metpy.cbook import get_test_data
from metpy.units import units

For this, note that **any** import of `metpy` will enable access to the xarray accessors, which we'll cover below.

## Pint

The unit registry encapsulates all of the available units, as well as any pertinent settings.
The registry also understands unit prefixes and suffixes; this allows the registry to
understand `'kilometer'` and `'meters'` in addition to the base `'meter'` unit.

In general, using units is only a small step on top of using the `numpy.ndarray`
object.

### Adding Units to Data
The easiest way to attach units to an array (or integer, float, etc.) is to multiply by the
units:



In [2]:
distance = np.arange(1, 5) * units.meters
distance

Magnitude,[1 2 3 4]
Units,meter


It is also possible to directly construct a `pint.Quantity`, with a full units string:



In [3]:
time = units.Quantity(np.arange(2, 10, 2), 'sec')

Compound units can be constructed by the direct mathematical operations necessary:



In [4]:
9.81 * units.meter / (units.second * units.second)

9.81 <Unit('meter / second ** 2')>

This verbose syntax can be reduced by using the unit registry's support for parsing units:



In [5]:
9.81 * units('m/s^2')

9.81 <Unit('meter / second ** 2')>

### Operations With Units
With units attached, it is possible to perform mathematical operations, resulting in the
proper units:



In [6]:
distance / time

Magnitude,[0.5 0.5 0.5 0.5]
Units,meter/second


For multiplication and division, units can combine and cancel. For addition and subtraction,
instead the operands must have compatible units. For instance, this works:



In [7]:
distance + distance

Magnitude,[2 4 6 8]
Units,meter


However...

In [8]:
distance + time

DimensionalityError: Cannot convert from 'meter' ([length]) to 'second' ([time])

would not.

Even if the units are not identical, as long as they are dimensionally equivalent, the
operation can be performed:

In [9]:
3 * units.inch + 5 * units.cm

4.968503937007874 <Unit('inch')>

### Converting Units

Converting a `pint.Quantity` between units can be accomplished by using the
`pint.Quantity.to` method call, which constructs a new `pint.Quantity` in the
desired units:



In [10]:
(1 * units.inch).to(units.mm)

25.4 <Unit('millimeter')>

There is also the `pint.Quantity.ito` method which performs the same operation
in-place:



In [11]:
a = np.arange(5.) * units.meter
a.ito('feet')
a

Magnitude,[0.0 3.2808398950131235 6.561679790026247 9.84251968503937 13.123359580052494]
Units,foot


To simplify units, there is also the `pint.Quantity.to_base_units` method,
which converts a quantity to SI units, performing any needed cancellation:



In [12]:
Lf = 3.34e6 * units('J/kg')
Lf, Lf.to_base_units()

(3340000.0 <Unit('joule / kilogram')>,
 3340000.0 <Unit('meter ** 2 / second ** 2')>)

`pint.Quantity.to_base_units` can also be done in-place via the
`pint.Quantity.ito_base_units` method.

By default Pint does not do any more than simple unit simplification, so when you perform
operations you could get some seemingly odd results:



In [13]:
length = 10.4 * units.inch
width = 5 * units.cm
area = length * width
area

52.0 <Unit('centimeter * inch')>

This is another place where `pint.Quantity.to` comes in handy:



In [14]:
area.to('m^2')

0.013208 <Unit('meter ** 2')>

### Units in MetPy calculations

In [15]:
temperature = 73.2 * units.degF
rh = 64 * units.percent
dewpoint = mpcalc.dewpoint_from_relative_humidity(temperature, rh)
dewpoint

15.726236381245258 <Unit('degree_Celsius')>

### Headaches?
Masked arrays! Left and right multiplication? Manual `Quantity` construction, `mb` vs `mbar`

## Xarray

MetPy also provides DataArray and Dataset
*accessors* (collections of methods and properties attached to the `.metpy` property for
coordinate/CRS and unit operations.

Let's open some sample data to work with.



In [16]:
# Open the netCDF file as a xarray Dataset
data = xr.open_dataset(get_test_data('irma_gfs_example.nc', False))

# View a summary of the Dataset
data

<xarray.Dataset>
Dimensions:                              (time1: 9, latitude: 81,
                                          isobaric3: 31, isobaric1: 21,
                                          longitude: 131)
Coordinates:
  * time1                                (time1) datetime64[ns] 2017-09-05T12...
    reftime                              datetime64[ns] ...
  * latitude                             (latitude) float32 50.0 49.5 ... 10.0
  * isobaric3                            (isobaric3) float64 100.0 ... 1e+05
  * isobaric1                            (isobaric1) float64 1e+04 ... 1e+05
  * longitude                            (longitude) float32 250.0 ... 315.0
Data variables:
    Vertical_velocity_pressure_isobaric  (time1, isobaric1, latitude, longitude) float32 ...
    Relative_humidity_isobaric           (time1, isobaric3, latitude, longitude) float32 ...
    Temperature_isobaric                 (time1, isobaric3, latitude, longitude) float32 ...
    u-component_of_wind_isobaric         (time1, isobaric3, latitude, longitude) float32 ...
    v-component_of_wind_isobaric         (time1, isobaric3, latitude, longitude) float32 ...
    Geopotential_height_isobaric         (time1, isobaric3, latitude, longitude) float32 ...
    LatLon_361X720-0p25S-180p00E         int32 ...
Attributes: (12/13)
    Originating_or_generating_Center:                                        ...
    Originating_or_generating_Subcenter:                                     ...
    GRIB_table_version:                                                      ...
    Type_of_generating_process:                                              ...
    Analysis_or_forecast_generating_process_identifier_defined_by_originating...
    Conventions:                                                             ...
    ...                                                                                ...
    featureType:                                                             ...
    History:                                                                 ...
    geospatial_lat_min:                                                      ...
    geospatial_lat_max:                                                      ...
    geospatial_lon_min:                                                      ...
    geospatial_lon_max:                                                      ...

In [17]:
temperature = data['Temperature_isobaric']
temperature

<xarray.DataArray 'Temperature_isobaric' (time1: 9, isobaric3: 31,
                                          latitude: 81, longitude: 131)>
[2960469 values with dtype=float32]
Coordinates:
  * time1      (time1) datetime64[ns] 2017-09-05T12:00:00 ... 2017-09-06T12:0...
    reftime    datetime64[ns] 2017-09-05T12:00:00
  * latitude   (latitude) float32 50.0 49.5 49.0 48.5 ... 11.5 11.0 10.5 10.0
  * isobaric3  (isobaric3) float64 100.0 200.0 300.0 ... 9.5e+04 9.75e+04 1e+05
  * longitude  (longitude) float32 250.0 250.5 251.0 251.5 ... 314.0 314.5 315.0
Attributes:
    long_name:                      Temperature @ Isobaric surface
    units:                          K
    Grib_Variable_Id:               VAR_0-0-0_L100
    Grib2_Parameter:                [0 0 0]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Temperature
    Grib2_Parameter_Name:           Temperature
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

### Coordinates and Coordinate Reference Systems

MetPy's first set of helpers comes with identifying *coordinate types*. In a given dataset,
coordinates can have a variety of different names and yet refer to the same type (such as
"isobaric1" and "isobaric3" both referring to vertical isobaric coordinates). Following
CF conventions, as well as using some fall-back regular expressions, MetPy can
systematically identify coordinates of the following types:

- time
- vertical
- latitude
- y
- longitude
- x

When identifying a single coordinate, it is best to use the property directly associated
with that type



In [18]:
temperature.metpy.time

<xarray.DataArray 'time1' (time1: 9)>
array(['2017-09-05T12:00:00.000000000', '2017-09-05T15:00:00.000000000',
       '2017-09-05T18:00:00.000000000', '2017-09-05T21:00:00.000000000',
       '2017-09-06T00:00:00.000000000', '2017-09-06T03:00:00.000000000',
       '2017-09-06T06:00:00.000000000', '2017-09-06T09:00:00.000000000',
       '2017-09-06T12:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time1    (time1) datetime64[ns] 2017-09-05T12:00:00 ... 2017-09-06T12:00:00
    reftime  datetime64[ns] 2017-09-05T12:00:00
Attributes:
    standard_name:  time
    long_name:      time
    udunits:        Hour since 2017-09-05T12:00:00Z
    _metpy_axis:    time

When accessing multiple coordinate types simultaneously, you can use the ``.coordinates()``
method to yield a generator for the respective coordinates



In [19]:
x, y = temperature.metpy.coordinates('x', 'y')
x

<xarray.DataArray 'longitude' (longitude: 131)>
array([250. , 250.5, 251. , 251.5, 252. , 252.5, 253. , 253.5, 254. , 254.5,
       255. , 255.5, 256. , 256.5, 257. , 257.5, 258. , 258.5, 259. , 259.5,
       260. , 260.5, 261. , 261.5, 262. , 262.5, 263. , 263.5, 264. , 264.5,
       265. , 265.5, 266. , 266.5, 267. , 267.5, 268. , 268.5, 269. , 269.5,
       270. , 270.5, 271. , 271.5, 272. , 272.5, 273. , 273.5, 274. , 274.5,
       275. , 275.5, 276. , 276.5, 277. , 277.5, 278. , 278.5, 279. , 279.5,
       280. , 280.5, 281. , 281.5, 282. , 282.5, 283. , 283.5, 284. , 284.5,
       285. , 285.5, 286. , 286.5, 287. , 287.5, 288. , 288.5, 289. , 289.5,
       290. , 290.5, 291. , 291.5, 292. , 292.5, 293. , 293.5, 294. , 294.5,
       295. , 295.5, 296. , 296.5, 297. , 297.5, 298. , 298.5, 299. , 299.5,
       300. , 300.5, 301. , 301.5, 302. , 302.5, 303. , 303.5, 304. , 304.5,
       305. , 305.5, 306. , 306.5, 307. , 307.5, 308. , 308.5, 309. , 309.5,
       310. , 310.5, 311. , 311.5, 312. , 312.5, 313. , 313.5, 314. , 314.5,
       315. ], dtype=float32)
Coordinates:
    reftime    datetime64[ns] 2017-09-05T12:00:00
  * longitude  (longitude) float32 250.0 250.5 251.0 251.5 ... 314.0 314.5 315.0
Attributes:
    units:          degrees_east
    standard_name:  longitude
    _metpy_axis:    x,longitude

These coordinate type aliases can also be used in MetPy's wrapped `.sel` and `.loc`
for indexing and selecting on `DataArray`s. For example, to access 500 hPa heights at
1800Z,

In [20]:
heights = data['Geopotential_height_isobaric'].metpy.sel(
    time='2017-09-05 18:00',
    vertical=50000.
)

Beyond just the coordinates themselves, a common need for both calculations with and plots
of geospatial data is knowing the coordinate reference system (CRS) on which the horizontal
spatial coordinates are defined. MetPy follows the [CF Conventions](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.8/cf-conventions.html#grid-mappings-and-projections)
for its CRS definitions, which it then caches on the `metpy_crs` coordinate in order for
it to persist through calculations and other array operations. There are two ways to do so
in MetPy:

First, if your dataset is already conforming to the CF Conventions, it will have a grid
mapping variable that is associated with the other data variables by the `grid_mapping`
attribute. This is automatically parsed via the `.parse_cf()` method:



In [21]:
# Parse full dataset
data_parsed = data.metpy.parse_cf()

# Parse subset of dataset
data_subset = data.metpy.parse_cf([
    'u-component_of_wind_isobaric',
    'v-component_of_wind_isobaric',
    'Vertical_velocity_pressure_isobaric'
])

# Parse single variable
relative_humidity = data.metpy.parse_cf('Relative_humidity_isobaric')

If your dataset doesn't have a CF-conforming grid mapping variable, you can manually specify
the CRS using the ``.assign_crs()`` method:



In [22]:
temperature = data['Temperature_isobaric'].metpy.assign_crs(
    grid_mapping_name='latitude_longitude',
    earth_radius=6371229.0
)

temperature

<xarray.DataArray 'Temperature_isobaric' (time1: 9, isobaric3: 31,
                                          latitude: 81, longitude: 131)>
[2960469 values with dtype=float32]
Coordinates:
  * time1      (time1) datetime64[ns] 2017-09-05T12:00:00 ... 2017-09-06T12:0...
    reftime    datetime64[ns] 2017-09-05T12:00:00
  * latitude   (latitude) float32 50.0 49.5 49.0 48.5 ... 11.5 11.0 10.5 10.0
  * isobaric3  (isobaric3) float64 100.0 200.0 300.0 ... 9.5e+04 9.75e+04 1e+05
  * longitude  (longitude) float32 250.0 250.5 251.0 251.5 ... 314.0 314.5 315.0
    metpy_crs  object Projection: latitude_longitude
Attributes:
    long_name:                      Temperature @ Isobaric surface
    units:                          K
    Grib_Variable_Id:               VAR_0-0-0_L100
    Grib2_Parameter:                [0 0 0]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Temperature
    Grib2_Parameter_Name:           Temperature
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

Notice the newly added ``metpy_crs`` non-dimension coordinate. Now how can we use this in
practice? For individual ``DataArrays``\s, we can access the cartopy and pyproj objects
corresponding to this CRS:



In [23]:
# Cartopy CRS, useful for plotting
relative_humidity.metpy.cartopy_crs

<Derived Projected CRS: +proj=eqc +ellps=WGS84 +a=6378137.0 +lon_0=0.0 +to ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (unknown)
- N[north]: Northing (unknown)
- h[up]: Ellipsoidal height (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Equidistant Cylindrical
Datum: unknown
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [24]:
# pyproj CRS, useful for projection transformations and forward/backward azimuth and great
# circle calculations
temperature.metpy.pyproj_crs

<Geographic 2D CRS: {"$schema": "https://proj.org/schemas/v0.2/projjso ...>
Name: undefined
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: undefined
- Ellipsoid: undefined
- Prime Meridian: Greenwich

### Units

Since unit-aware calculations are a major part of the MetPy library, unit support is a major
part of MetPy's xarray integration!

One very important point of consideration is that xarray data variables (in both
`Dataset`s and `DataArray`s) can store both unit-aware and unit-naive array types.
Unit-naive array types will be used by default in xarray, so we need to convert to a
unit-aware type if we want to use xarray operations while preserving unit correctness. MetPy
provides the `.quantify()` method for this.

In [25]:
heights = heights.metpy.quantify()
heights

Magnitude,[[5883.3564453125 5879.31640625 5875.3564453125 ... 5769.396484375 5769.99609375 5770.43603515625] [5885.47607421875 5882.236328125 5877.67626953125 ... 5783.916015625 5784.51611328125 5785.1962890625] [5888.87646484375 5885.15625 5880.916015625 ... 5798.47607421875 5799.076171875 5799.71630859375] ... [5892.51611328125 5892.236328125 5891.99609375 ... 5881.55615234375 5881.43603515625 5880.59619140625] [5891.83642578125 5891.59619140625 5891.71630859375 ... 5880.2763671875 5880.3564453125 5879.916015625] [5891.3564453125 5891.59619140625 5891.3564453125 ... 5879.87646484375 5879.59619140625 5878.99609375]]
Units,meter


Notice how the units are now represented in the data itself, rather than as a text
attribute. Now, even if we perform some kind of xarray operation (such as taking the zonal
mean), the units are preserved



In [26]:
heights_mean = heights.mean('longitude')
heights_mean

Magnitude,[5636.3671875 5644.39208984375 5652.7705078125 5661.369140625 5669.81298828125 5678.50830078125 5687.43310546875 5696.51708984375 5705.9267578125 5715.6123046875 5725.7021484375 5736.138671875 5746.8837890625 5757.7177734375 5768.56591796875 5779.28125 5789.8193359375 5799.80078125 5809.501953125 5818.771484375 5827.7744140625 5836.9208984375 5846.3857421875 5855.66796875 5864.23974609375 5871.80419921875 5878.76904296875 5885.05810546875 5890.12890625 5894.029296875 5897.14599609375 5899.6328125 5901.673828125 5903.27734375 5904.41015625 5905.27001953125 5905.89697265625 5906.591796875 5907.00830078125 5907.08056640625 5907.03076171875 5906.87109375 5906.77685546875 5906.45556640625 5906.12109375 5905.7978515625 5905.6826171875 5905.5986328125 5905.40966796875 5905.12451171875 5904.77294921875 5904.43017578125 5903.9541015625 5903.3779296875 5902.5322265625 5901.7138671875 5900.865234375 5900.09326171875 5899.3359375 5898.46728515625 5897.43994140625 5896.23291015625 5895.05712890625 5893.6328125 5891.5322265625 5888.14990234375 5883.09423828125 5884.80224609375 5887.9228515625 5889.00537109375 5889.11376953125 5888.888671875 5888.68505859375 5888.14599609375 5887.56689453125 5887.03173828125 5886.298828125 5885.42138671875 5884.51513671875 5883.69384765625 5883.07177734375]
Units,meter


However, this "quantification" is not without its consequences. By default, xarray loads its
data lazily to conserve memory usage. Unless your data is chunked into a Dask array (using
the ``chunks`` argument), this ``.quantify()`` method will load data into memory, which
could slow your script or even cause your process to run out of memory. And so, we recommend
subsetting your data before quantifying it.

Also, these Pint ``Quantity`` data objects are not properly handled by xarray when writing
to disk. And so, if you want to safely export your data, you will need to undo the
quantification with the ``.dequantify()`` method, which converts your data back to a
unit-naive array with the unit as a text attribute



In [27]:
heights_mean_str_units = heights_mean.metpy.dequantify()
heights_mean_str_units

<xarray.DataArray 'Geopotential_height_isobaric' (latitude: 81)>
array([5636.367 , 5644.392 , 5652.7705, 5661.369 , 5669.813 , 5678.5083,
       5687.433 , 5696.517 , 5705.927 , 5715.6123, 5725.702 , 5736.1387,
       5746.884 , 5757.718 , 5768.566 , 5779.2812, 5789.8193, 5799.801 ,
       5809.502 , 5818.7715, 5827.7744, 5836.921 , 5846.3857, 5855.668 ,
       5864.2397, 5871.804 , 5878.769 , 5885.058 , 5890.129 , 5894.0293,
       5897.146 , 5899.633 , 5901.674 , 5903.2773, 5904.41  , 5905.27  ,
       5905.897 , 5906.592 , 5907.0083, 5907.0806, 5907.031 , 5906.871 ,
       5906.777 , 5906.4556, 5906.121 , 5905.798 , 5905.6826, 5905.5986,
       5905.4097, 5905.1245, 5904.773 , 5904.43  , 5903.954 , 5903.378 ,
       5902.532 , 5901.714 , 5900.865 , 5900.0933, 5899.336 , 5898.4673,
       5897.44  , 5896.233 , 5895.057 , 5893.633 , 5891.532 , 5888.15  ,
       5883.094 , 5884.8022, 5887.923 , 5889.0054, 5889.114 , 5888.8887,
       5888.685 , 5888.146 , 5887.567 , 5887.0317, 5886.299 , 5885.4214,
       5884.515 , 5883.694 , 5883.072 ], dtype=float32)
Coordinates:
    time1      datetime64[ns] 2017-09-05T18:00:00
    reftime    datetime64[ns] 2017-09-05T12:00:00
  * latitude   (latitude) float32 50.0 49.5 49.0 48.5 ... 11.5 11.0 10.5 10.0
    isobaric3  float64 5e+04
Attributes:
    units:    meter

Other useful unit integration features include:

Unit-based selection/indexing:



In [28]:
heights_at_45_north = data['Geopotential_height_isobaric'].metpy.sel(
    latitude=45 * units.degrees_north,
    vertical=300 * units.hPa
)
heights_at_45_north

<xarray.DataArray 'Geopotential_height_isobaric' (time1: 9, longitude: 131)>
array([[9682.115, 9676.315, 9673.275, ..., 9608.755, 9609.035, 9609.395],
       [9683.031, 9678.352, 9675.312, ..., 9623.551, 9624.352, 9625.071],
       [9688.037, 9685.677, 9681.677, ..., 9631.757, 9632.437, 9633.157],
       ...,
       [9677.054, 9678.254, 9669.454, ..., 9656.294, 9656.054, 9655.894],
       [9665.767, 9666.247, 9658.087, ..., 9666.047, 9665.487, 9664.887],
       [9660.616, 9659.856, 9652.017, ..., 9681.176, 9680.097, 9679.097]],
      dtype=float32)
Coordinates:
  * time1      (time1) datetime64[ns] 2017-09-05T12:00:00 ... 2017-09-06T12:0...
    reftime    datetime64[ns] 2017-09-05T12:00:00
    latitude   float32 45.0
    isobaric3  float64 3e+04
  * longitude  (longitude) float32 250.0 250.5 251.0 251.5 ... 314.0 314.5 315.0
Attributes:
    long_name:                      Geopotential height @ Isobaric surface
    units:                          gpm
    Grib_Variable_Id:               VAR_0-3-5_L100
    Grib2_Parameter:                [0 3 5]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Mass
    Grib2_Parameter_Name:           Geopotential height
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

Unit conversion:



In [29]:
temperature_degc = temperature[0].metpy.convert_units('degC')
temperature_degc

<xarray.DataArray 'Temperature_isobaric' (isobaric3: 31, latitude: 81,
                                          longitude: 131)>
<Quantity([[[-15.149994 -15.149994 -15.25     ... -14.949982 -14.949982 -14.850006]
  [-15.25     -15.25     -15.350006 ... -14.949982 -14.949982 -14.949982]
  [-15.350006 -15.350006 -15.350006 ... -15.049988 -15.049988 -15.049988]
  ...
  [-12.350006 -12.350006 -12.350006 ... -15.149994 -15.049988 -14.949982]
  [-12.149994 -12.149994 -12.149994 ... -15.049988 -15.049988 -14.949982]
  [-11.949982 -11.949982 -11.949982 ... -15.049988 -14.949982 -14.850006]]

 [[-16.649994 -16.649994 -16.649994 ... -17.549988 -17.549988 -17.649994]
  [-16.75     -16.75     -16.850006 ... -17.649994 -17.649994 -17.649994]
  [-16.949982 -16.949982 -16.949982 ... -17.75     -17.75     -17.75    ]
  ...
  [-14.049988 -14.049988 -14.049988 ... -12.049988 -11.75     -11.549988]
  [-13.949982 -13.949982 -13.949982 ... -12.049988 -11.75     -11.549988]
  [-13.75     -13.850006 -13.850006 ... -12.049988 -11.75     -11.549988]]

 [[-21.75     -21.75     -21.75     ... -22.649994 -22.649994 -22.649994]
  [-21.949997 -21.84999  -21.84999  ... -22.75     -22.75     -22.75    ]
  [-22.049988 -22.049988 -22.049988 ... -22.84999  -22.84999  -22.949997]
  ...
...
  ...
  [ 22.149994  22.149994  22.149994 ...  22.550018  22.649994  22.75    ]
  [ 22.050018  22.050018  21.950012 ...  22.550018  22.149994  22.350006]
  [ 21.950012  21.75      22.149994 ...  22.550018  22.450012  22.050018]]

 [[ 15.649994  15.149994  14.649994 ...  11.950012  11.850006  11.75    ]
  [ 15.149994  15.149994  15.050018 ...  11.950012  11.649994  11.450012]
  [ 15.050018  14.850006  15.050018 ...  11.850006  11.75      11.649994]
  ...
  [ 24.149994  24.25      24.149994 ...  24.649994  24.649994  24.850006]
  [ 23.950012  23.950012  23.850006 ...  24.649994  24.25      24.450012]
  [ 23.850006  23.649994  23.75     ...  24.649994  24.25      24.050018]]

 [[ 17.050018  16.550018  16.050018 ...  13.450012  13.450012  13.550018]
  [ 16.550018  16.550018  16.450012 ...  13.350006  13.050018  12.950012]
  [ 16.450012  16.25      16.450012 ...  13.050018  12.850006  12.850006]
  ...
  [ 26.350006  26.350006  26.25     ...  26.75      26.850006  26.950012]
  [ 26.050018  26.050018  25.950012 ...  26.75      26.350006  26.550018]
  [ 26.050018  25.75      25.649994 ...  26.75      26.350006  26.149994]]], 'degree_Celsius')>
Coordinates:
    time1      datetime64[ns] 2017-09-05T12:00:00
    reftime    datetime64[ns] 2017-09-05T12:00:00
  * latitude   (latitude) float32 50.0 49.5 49.0 48.5 ... 11.5 11.0 10.5 10.0
  * isobaric3  (isobaric3) float64 100.0 200.0 300.0 ... 9.5e+04 9.75e+04 1e+05
  * longitude  (longitude) float32 250.0 250.5 251.0 251.5 ... 314.0 314.5 315.0
    metpy_crs  object Projection: latitude_longitude
Attributes:
    long_name:                      Temperature @ Isobaric surface
    Grib_Variable_Id:               VAR_0-0-0_L100
    Grib2_Parameter:                [0 0 0]
    Grib2_Parameter_Discipline:     Meteorological products
    Grib2_Parameter_Category:       Temperature
    Grib2_Parameter_Name:           Temperature
    Grib2_Level_Type:               100
    Grib2_Level_Desc:               Isobaric surface
    Grib2_Generating_Process_Type:  Forecast
    grid_mapping:                   LatLon_361X720-0p25S-180p00E

Unit conversion for coordinates:



In [30]:
heights_on_hpa_levels = heights.metpy.convert_coordinate_units('isobaric3', 'hPa')
heights_on_hpa_levels['isobaric3']

<xarray.DataArray 'isobaric3' ()>
array(500.)
Coordinates:
    time1      datetime64[ns] 2017-09-05T18:00:00
    reftime    datetime64[ns] 2017-09-05T12:00:00
    isobaric3  float64 500.0
Attributes:
    units:        hPa
    positive:     down
    _metpy_axis:  vertical

Accessing just the underlying unit array:



In [31]:
heights_unit_array = heights.metpy.unit_array
heights_unit_array

Magnitude,[[5883.3564453125 5879.31640625 5875.3564453125 ... 5769.396484375 5769.99609375 5770.43603515625] [5885.47607421875 5882.236328125 5877.67626953125 ... 5783.916015625 5784.51611328125 5785.1962890625] [5888.87646484375 5885.15625 5880.916015625 ... 5798.47607421875 5799.076171875 5799.71630859375] ... [5892.51611328125 5892.236328125 5891.99609375 ... 5881.55615234375 5881.43603515625 5880.59619140625] [5891.83642578125 5891.59619140625 5891.71630859375 ... 5880.2763671875 5880.3564453125 5879.916015625] [5891.3564453125 5891.59619140625 5891.3564453125 ... 5879.87646484375 5879.59619140625 5878.99609375]]
Units,meter


In [32]:
heights.values

array([[5883.3564, 5879.3164, 5875.3564, ..., 5769.3965, 5769.996 ,
        5770.436 ],
       [5885.476 , 5882.2363, 5877.6763, ..., 5783.916 , 5784.516 ,
        5785.1963],
       [5888.8765, 5885.1562, 5880.916 , ..., 5798.476 , 5799.076 ,
        5799.7163],
       ...,
       [5892.516 , 5892.2363, 5891.996 , ..., 5881.556 , 5881.436 ,
        5880.596 ],
       [5891.8364, 5891.596 , 5891.7163, ..., 5880.2764, 5880.3564,
        5879.916 ],
       [5891.3564, 5891.596 , 5891.3564, ..., 5879.8765, 5879.596 ,
        5878.996 ]], dtype=float32)

Accessing just the underlying units:



In [33]:
height_units = heights.metpy.units
height_units

<Unit('meter')>

### Calculations

MetPy's xarray integration extends to its calculation suite as well. Most grid-capable
calculations (such as thermodynamics, kinematics, and smoothers) fully support xarray
``DataArray``\s by accepting them as inputs, returning them as outputs, and automatically
using the attached coordinate data/metadata to determine grid arguments



In [34]:
heights = data_parsed.metpy.parse_cf('Geopotential_height_isobaric').metpy.sel(
    time='2017-09-05 18:00',
    vertical=500 * units.hPa
)
u_g, v_g = mpcalc.geostrophic_wind(heights)
u_g

Magnitude,[[2.3353588579406686 4.60981877223844 2.9362541310137718 ... 22.890679520432215 22.891835829490446 23.49234566620837] [4.389668010973033 4.644000832484165 4.421119856074913 ... 23.124872038344872 23.125260332728804 23.284461030163367] [5.54483394787313 4.999466098910912 5.352351177650735 ... 22.979391208366312 22.94731074666258 22.754827976440804] ... [-3.4213423527971765 -3.296001452492959 -3.1691131336642333 ... -7.605562037088769 -6.844232124116416 -4.5648846407753245] [-3.8480188676824527 -2.1241064149587885 -2.122486196488429 ... -5.573551538876476 -6.104983197236323 -5.309455928166912] [-2.647433169930482 2.2291489311550605 -2.785160907090209 ... 0.27885615921944024 -4.182842387936929 -7.2417584269395885]]
Units,meter/second


A few remaining portions of MetPy's calculations (mainly the interpolation module and a few
other functions) do not fully support xarray, and so, use of `.values` may be needed to
convert to a bare NumPy array.

### CF-Compliant Dataset Example

The GFS sample used throughout this tutorial so far has been an example of a CF-compliant
dataset. These kinds of datasets are easiest to work with it MetPy, since most of the
"xarray magic" uses CF metadata. For this kind of dataset, a typical workflow looks like the
following

In [35]:
# Load data, parse it for a CF grid mapping, and promote lat/lon data variables to coordinates
data = xr.open_dataset(
    get_test_data('narr_example.nc', False)
).metpy.parse_cf().set_coords(['lat', 'lon'])

# Subset to only the data you need to save on memory usage
subset = data.metpy.sel(isobaric=850 * units.hPa).squeeze()

# Quantify if you plan on performing xarray operations that need to maintain unit correctness
subset = subset.metpy.quantify()

In [36]:
temp = subset.Temperature.metpy.unit_array
u = subset.u_wind.metpy.unit_array
v = subset.v_wind.metpy.unit_array
lon = subset.lon.metpy.unit_array
lat = subset.lat.metpy.unit_array

temp

Magnitude,[[284.8985595703125 284.8985595703125 284.8985595703125 ... 292.6329345703125 292.7579345703125 293.0548095703125] [284.8829345703125 284.9298095703125 284.8985595703125 ... 292.3048095703125 292.4141845703125 292.4141845703125] [284.8673095703125 284.8985595703125 284.8985595703125 ... 291.9141845703125 291.9141845703125 291.9298095703125] ... [271.2110595703125 271.0079345703125 270.8360595703125 ... 280.8516845703125 280.8985595703125 280.9298095703125] [271.1641845703125 270.9454345703125 270.7735595703125 ... 280.8673095703125 280.9141845703125 280.9923095703125] [271.1016845703125 270.9454345703125 270.8204345703125 ... 280.8673095703125 280.9141845703125 281.0391845703125]]
Units,kelvin


In [37]:
mpcalc.lat_lon_grid_deltas?

Signature:
mpcalc.lat_lon_grid_deltas(
    longitude,
    latitude,
    x_dim=-1,
    y_dim=-2,
    geod=None,
)
Docstring:
Calculate the actual delta between grid points that are in latitude/longitude format.

Parameters
----------
longitude : array_like
    Array of longitudes defining the grid. If not a `pint.Quantity`, assumed to be in
    degrees.

latitude : array_like
    Array of latitudes defining the grid. If not a `pint.Quantity`, assumed to be in
    degrees.
x_dim: int
    axis number for the x dimension, defaults to -1.
y_dim : int
    axis number for the y dimension, defaults to -2.
geod : `pyproj.Geod` or ``None``
    PyProj Geod to use for forward azimuth and distance calculations. If ``None``, use a
    default spherical ellipsoid.

Returns
-------
dx, dy:
    At least two dimensional arrays of signed deltas between grid points in the x and y
    direction

Notes
-----
Accepts 1D, 2D, or higher arrays for latitude and longitude
Assumes [..., Y, X] dimension order for 

In [38]:
dx, dy = mpcalc.lat_lon_grid_deltas(lon, lat)

In [39]:
mpcalc.advection?

Signature:
mpcalc.advection(
    scalar,
    u=None,
    v=None,
    w=None,
    *,
    dx=None,
    dy=None,
    dz=None,
    x_dim=-1,
    y_dim=-2,
    vertical_dim=-3,
)
Docstring:
Calculate the advection of a scalar field by the wind.

Parameters
----------
scalar : `pint.Quantity` or `xarray.DataArray`
    Array (with N-dimensions) with the quantity to be advected. Use `xarray.DataArray` to
    have dimension ordering automatically determined, otherwise, use default
    [..., Z, Y, X] ordering or specify \*_dim keyword arguments.
u, v, w : `pint.Quantity` or `xarray.DataArray` or None
    N-dimensional arrays with units of velocity representing the flow, with a component of
    the wind in each dimension. For 1D advection, use 1 positional argument (with `dx` for
    grid spacing and `x_dim` to specify axis if not the default of -1) or use 1 applicable
    keyword argument (u, v, or w) for proper physical dimension (with corresponding `d\*`
    for grid spacing and `\*_dim` to sp

In [40]:
mpcalc.advection(temp, u, v, w=None, dx=dx, dy=dy, dz=None, x_dim=-1, y_dim=-2)

Magnitude,[[-2.429513110314247e-06 9.40965414630125e-06 -2.204242700734707e-06 ... 1.6078958845501666e-05 2.05737318924406e-05 6.837107114446457e-05] [2.4191817303088816e-05 2.14672783536173e-06 -1.0242993005501698e-05 ... 1.3850261153823093e-05 3.336890667639327e-05 3.7508108080389135e-05] [1.7312319985026526e-05 -1.513340034418501e-06 -4.459454440602606e-06 ... 9.644809727119217e-07 1.708395891073388e-05 4.661772589002127e-05] ... [8.707740379274563e-05 8.442435616305721e-05 7.583857065113922e-05 ... 6.425158490214866e-06 3.27131760846954e-06 5.553129525489943e-06] [9.26568966214204e-05 7.122247224034762e-05 4.767891393797653e-05 ... 1.6445801513928905e-06 1.4943267587372267e-06 6.688015726151595e-06] [8.125613337733372e-05 2.7245841221124756e-05 7.311234083087967e-07 ... -4.1411223432490034e-07 5.090062112380679e-06 8.696360555246224e-06]]
Units,kelvin/second


or...

In [41]:
mpcalc.advection(subset.Temperature, subset.u_wind, subset.v_wind)

/home/dcamron/mambaforge/envs/pythia-metpy-tutorial/lib/python3.10/site-packages/metpy/xarray.py:1468: UserWarning: Vertical dimension number not found. Defaulting to (..., Z, Y, X) order.
  warnings.warn(


Magnitude,[[-2.4308587578508167e-06 9.414865922332617e-06 -2.205463576574719e-06 ... 1.6087864585374502e-05 2.0585127176558957e-05 6.840894019931781e-05] [2.420521655575422e-05 2.147916855123707e-06 -1.024866634743468e-05 ... 1.385793247285639e-05 3.3387388893158335e-05 3.752888289901886e-05] [1.732190885329267e-05 -1.5141782362476878e-06 -4.461924422752406e-06 ... 9.650151750036073e-07 1.709342129542926e-05 4.664354630191932e-05] ... [8.712563382520431e-05 8.447111673765669e-05 7.588057577031059e-05 ... 6.4287172274921895e-06 3.273129510832782e-06 5.556205267366144e-06] [9.270821699775316e-05 7.126192061614284e-05 4.770532211581401e-05 ... 1.645491043893071e-06 1.4951544295745315e-06 6.691720053582986e-06] [8.130113915116966e-05 2.7260932022550727e-05 7.315283595700546e-07 ... -4.143416009083011e-07 5.0928813726024214e-06 8.701177255490486e-06]]
Units,kelvin/second


In [42]:
# Save output
subset.metpy.dequantify().drop_vars('metpy_crs').to_netcdf('500hPa_analysis.nc')

### Non-Compliant Dataset Example

When CF metadata (such as grid mapping, coordinate attributes, etc.) are missing, a bit more
work is required to manually supply the required information, for example,



In [43]:
nonstandard = xr.Dataset({
    'temperature': (('y', 'x'), np.arange(0, 9).reshape(3, 3) * units.degC),
    'y': ('y', np.arange(0, 3) * 1e5, {'units': 'km'}),
    'x': ('x', np.arange(0, 3) * 1e5, {'units': 'km'})
})

# Add both CRS and then lat/lon coords using chained methods
data = nonstandard.metpy.assign_crs(
    grid_mapping_name='lambert_conformal_conic',
    latitude_of_projection_origin=38.5,
    longitude_of_central_meridian=262.5,
    standard_parallel=38.5,
    earth_radius=6371229.0
).metpy.assign_latitude_longitude()

# Preview the changes
data

Magnitude,[[0 1 2] [3 4 5] [6 7 8]]
Units,degree_Celsius
